Notebook author: Özgün Haznedar

 This notebook is used to resize and scale the landsat and sentinel TIF images downloaded from Google Earth Engine, plotting the results. The output is to be used by GDAL_transformer_PNG.

In [ ]:
import gdal
gdal.UseExceptions()
import matplotlib.pyplot as plt
import numpy as np
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# WRITE DATA DIRECTORY NAME
dataset_directory = 'PATH TO FINAL DATASET'

# DICTIONARY OF DATA SETS ( NAME : OUTPUT IMAGE SIZE)
dataset_dict = {"train_LR":265 , "val_LR":265 , "train_HR":795 ,"val_HR":795 } 

# OUTPUT FOLDER NAME
output_folder = "processed_manual_strech"

In [ ]:
for dataset , image_size in dataset_dict.items():

  source_path = os.path.join(dataset_directory, dataset)

  # iterate over files in
  # that directory
  file_dict = dict()
  for filename in os.listdir(source_path):
      f = os.path.join(source_path, filename)
      # checking if it is a file
      if os.path.isfile(f) and filename.split(".")[-1] == "tif":
        file_dict[filename] = f


  # create target folder
  directory = output_folder+"/"+dataset
  output_path = os.path.join(dataset_directory, directory)

  os.makedirs(output_path)

  # iterate over all the files for transformation
  for file_name , file_path in file_dict.items():
    img_src  = file_path
    img_trg  = os.path.join(output_path, file_name) 

    gdal.Translate(img_trg, img_src, width=image_size, height=image_size)
    


In [ ]:
output_paths = list()
for dataset , image_size in dataset_dict.items():
  directory = output_folder+"/"+dataset
  output_path = os.path.join(dataset_directory, directory)  
  output_paths.append(output_path)

for folder in output_paths:
  if "LR" in folder:
    for filename in os.listdir(folder):
      f = os.path.join(folder, filename)
      
      ds = gdal.Open(f, gdal.GA_Update)
      data = ds.ReadAsArray()

      lr_image = ((data*0.0000275-0.2)*255*4).astype(int) # landsat conversion original , *4 is manual correction
      lr_image[lr_image>255] = 255
      lr_image[lr_image<0] = 0

      for band in range(3):
          ds.GetRasterBand(band+1).WriteArray(lr_image[band, :, :]) 
      ds = None      

  elif "HR" in folder:
    for filename in os.listdir(folder):
      f = os.path.join(folder, filename)
      
      ds = gdal.Open(f, gdal.GA_Update)
      data = ds.ReadAsArray()

      hr_image = (data*(255/3558)*1.4).astype(int) # sentinel conversion , *1.4 is manual correction      
      hr_image[hr_image>255] = 255
      hr_image[hr_image<0] = 0
      
      for band in range(3):
          ds.GetRasterBand(band+1).WriteArray(hr_image[band, :, :]) 
      ds = None      


In [ ]:
# define the folders to be viewed
output_paths = list()
for dataset , image_size in dataset_dict.items():
  directory = output_folder+"/"+dataset
  output_path = os.path.join(dataset_directory, directory)  
  output_paths.append(output_path)

directory_LR = output_paths[1] #gets the val_LR folder
directory_HR = output_paths[3] #gets the val_HR folder

# iterate over files in
# that directories
file_dict = dict()
for filename in os.listdir(directory_LR):
    f_lr = os.path.join(directory_LR, filename)
    f_hr = os.path.join(directory_HR, filename)
    # checking if it is a file
    if os.path.isfile(f_lr):
      file_dict[f_lr] = f_hr

for idx , (lr,hr) in enumerate(file_dict.items()):
  lr = gdal.Open(lr)
  lr = lr.ReadAsArray()
  lr = np.rollaxis(lr,0,3)
  hr = gdal.Open(hr)
  hr = hr.ReadAsArray()
  hr = np.rollaxis(hr,0,3)
  fig, ax = plt.subplots(1,2,figsize=(15,15))
  ax[0].imshow(lr)
  ax[0].set_title(f"landsat")
  ax[1].imshow(hr)
  ax[1].set_title(f"sentinel")
  plt.show()
  


NOTEBOOK FINISHED